<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Optimasi_Mebel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI")
#install.packages("ompr")
#install.packages("ompr.roi")
#install.packages("ROI.plugin.glpk")

In [16]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)
library(ggplot2)

rm(list=ls())

In [17]:
problem = 
data.frame(
  keterangan = c("meja","lemari","kursi", "man days"),
  workshop_1 = c("2 man days","5 man days","1 man days","10 man days per week"),
  workshop_2 = c("1 man days","3 man days","1 man days","14 man days"),
  demand_perweeks = c(5,3,6,""),
  profit = c(150,250,100,"")
)

problem

keterangan,workshop_1,workshop_2,demand_perweeks,profit
<chr>,<chr>,<chr>,<chr>,<chr>
meja,2 man days,1 man days,5,150
lemari,5 man days,3 man days,3,250
kursi,1 man days,1 man days,6,100
man days,10 man days per week,14 man days,,


In [18]:
# misalkan:
# i : banyak produk, i=1,2,3
# j : banyak plant, j=1,2

# x[i,j] banyaknya produk i yang diproduksi di plant j
# profit[i] : profit pada produk i

In [19]:
profit = c(150,250,100)

model = 
  MIPModel() %>%
  add_variable(x[i,j],
               i = 1:3,
               j = 1:2,
               type = "integer",
               lb = 0) %>%
  set_objective(sum_expr(x[i,j] * profit[i], i = 1:3,j = 1:2),
                "max"
               ) %>%
  add_constraint(2 * x[1,1] + 5 * x[2,1] + x[3,1] <= 10) %>%
  add_constraint(1 * x[1,2] + 3 * x[2,2] + x[3,2] <= 14) %>%
  add_constraint(x[1,1] + x[1,2] >= 5) %>%
  add_constraint(x[2,1] + x[2,2] >= 3) %>%
  add_constraint(x[3,1] + x[3,2] >= 6)
model


Mixed integer linear optimization problem
Variables:
  Continuous: 0 
  Integer: 6 
  Binary: 0 
Model sense: maximize 
Constraints: 5 

In [20]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))

<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
5 rows, 6 columns, 12 non-zeros
      0: obj =  -0.000000000e+00 inf =   1.400e+01 (3)
      5: obj =   2.100000000e+03 inf =   0.000e+00 (0)
*     7: obj =   2.500000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
5 rows, 6 columns, 12 non-zeros
6 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     7: mip =     not found yet <=              +inf        (1; 0)
+     7: >>>>>   2.500000000e+03 <=   2.500000000e+03   0.0% (1; 0)
+     7: mip =   2.500000000e+03 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


In [21]:
result$solution

x[1,1] x[2,1] x[3,1] x[1,2] x[2,2] x[3,2] 
     0      0     10      5      3      0

In [22]:
result$objective_value

[1] 2500